# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

---

## <font face="仿宋">第四部分导读

&emsp;&emsp;<font face="仿宋">在案例的第二、三部分中，我们详细介绍了关于特征工程的各项技术，特征工程技术按照大类来分可以分为数据预处理、特征衍生、特征筛选三部分，其中特征预处理的目的是为了将数据集整理、清洗到可以建模的程度，具体技术包括缺失值处理、异常值处理、数据重编码等，是建模之前必须对数据进行的处理和操作；而特征衍生和特征筛选则更像是一类优化手段，能够帮助模型突破当前数据集建模的效果上界。并且我们在第二部分完整详细的介绍机器学习可解释性模型的训练、优化和解释方法，也就是逻辑回归和决策树模型。并且此前我们也一直以这两种算法为主，来进行各个部分的模型测试。

&emsp;&emsp;<font face="仿宋">而第四部分，我们将开始介绍集成学习的训练和优化的实战技巧，尽管从可解释性角度来说，集成学习的可解释性并不如逻辑回归和决策树，但在大多数建模场景下，集成学习都将获得一个更好的预测结果，这也是目前效果优先的建模场景下最常使用的算法。

&emsp;&emsp;<font face="仿宋">总的来说，本部分内容只有一个目标，那就是借助各类优化方法，抵达每个主流集成学习的效果上界。换而言之，本部分我们将围绕单模优化策略展开详细的探讨，涉及到的具体集成学习包括随机森林、XGBoost、LightGBM、和CatBoost等目前最主流的集成学习算法，而具体的优化策略则包括超参数优化器的使用、特征衍生和筛选方法的使用、单模型自融合方法的使用，这些优化方法也是截至目前，提升单模效果最前沿、最有效、同时也是最复杂的方法。其中有很多较为艰深的理论，也有很多是经验之谈，但无论如何，我们希望能够围绕当前数据集，让每个集成学习算法优化到极限。值得注意的是，在这个过程中，我们会将此前介绍的特征衍生和特征筛选视作是一种模型优化方法，衍生和筛选的效果，一律以模型的最终结果来进行评定。而围绕集成学习进行海量特征衍生和筛选，也才是特征衍生和筛选技术能发挥巨大价值的主战场。

&emsp;&emsp;<font face="仿宋">而在抵达了单模的极限后，我们就会进入到下一阶段，也就是模型融合阶段。需要知道的是，只有单模的效果到达了极限，进一步的多模型融合、甚至多层融合，才是有意义的，才是有效果的。

---

# <center>Part 4.集成算法的训练与优化技巧

In [11]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 自定义模块
from telcoFunc import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc

&emsp;&emsp;然后执行Part 1中的数据清洗相关工作：

In [12]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [13]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

&emsp;&emsp;同时，创建自然编码后的数据集以及经过时序特征衍生的数据集：

In [14]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month']-1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month']-1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(enc.transform(X_train_seq).toarray(), 
                           columns = cate_colName(enc, seq_new, drop=None))

X_test_seq = pd.DataFrame(enc.transform(X_test_seq).toarray(), 
                          columns = cate_colName(enc, seq_new, drop=None))

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [15]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

然后是模型融合部分所需的第三方库、准备的数据以及训练好的模型：

In [16]:
# 本节新增第三方库
from joblib import dump, load
from sklearn.ensemble import VotingClassifier
from hyperopt import hp, fmin, tpe
from numpy.random import RandomState
from sklearn.model_selection import cross_val_score

In [17]:
logistic_search = load('logistic_search.joblib') 
tree_model = load('tree_model.joblib') 
RF_0 = load('RF_0.joblib') 

## <center>Ch.3 模型融合基础方法

## 二、Voting投票法与Averaging平均法

&emsp;&emsp;首先是较为基础的投票法和平均法，熟悉Bagging的大家肯定对这两类方法并不陌生，这类方法简单但同样实用，是模型融合实践过程中必不可少的融合方法。

&emsp;&emsp;首先我们先从这类方法的基础应用开始介绍，然后逐步深入。

### 1.基本方法介绍

- Voting投票法

&emsp;&emsp;首先是相对较为简单的模型融合方法：投票法与平均法。在深入学习了集成算法后，我们对投票法和均值法的计算过程并不陌生，我们可以简单回顾下投票法和平均法的集成过程，首先如果模型最终输出的是类别判别结果，则可以通过投票法进行模型融合，投票法会根据少数服从多数的规则进行结果输出，例如现有A、B、C三个模型对现有数据进行预测，结果如下：

<center><img src="https://s2.loli.net/2022/05/20/1JjNFKy8Uo4ftvW.png" alt="image-20220520153024128" style="zoom:33%;" />

很明显，根据少数如从多数的原则，投票得出三个样本的预测结果分别为1、0、1、0。

&emsp;&emsp;当然，很多时候为了计算方便，我们会把这个少数服从多数的过程等价转化是否超过半数评估器认为该样本应该属于1类，如果是，则输出结果为1，反之则输出预测结果为0。需要注意的是，该做法会更加方便代码层面的实现，也是后续我们主要采用的计算流程。例如上述简单示例可以修改流程如下：

<center><img src="https://s2.loli.net/2022/05/20/qY5UI3gDzSc8p2H.png" alt="image-20220520153651685" style="zoom:33%;" />

&emsp;&emsp;而这样的一个投票集成的过程，到底能带来多少性能上的提升呢？从理论上来说，根据[Narasimhamurthy，2003]研究表明，在多样性构建的比较好的情况下，投票融合性能边界如下：

<center><img src="https://s2.loli.net/2022/05/20/uVrzS79LaBsJgKp.png" alt="image-20220520122231347" style="zoom:33%;" />

能够看出，在单体分类器准确率为80%左右（较为普遍的情况）时，模型投票融合能有平均约15%的准确率提升。当然，该理论实际上是基于分类器相互独立的假设推导而来，而在大多数真实场景下，该假设并不成立，因此该理论的结论可以视作一个理论上限，并不能代表一般情况。

- Averaging平均法

&emsp;&emsp;而平均法，则是基于连续型预测结果进行融合，例如现在如果是回归类问题，三个模型对三个样本预测结果如下：

<center><img src="https://s2.loli.net/2022/05/16/xEtCmoWaV4AS3cj.png" alt="image-20220516172028022" style="zoom:33%;" />

则通过三个模型的预测结果进行平均，就能够得到最终模型融合的预测结果。

&emsp;&emsp;平均法的理论则可以通过更为简单的公式推导出来，一般来说，N个相互独立且误差为err的分类器进行简单平均法融合，最终得到的融合结果的误差为$\frac{1}{T}$。

- 概率均值法

&emsp;&emsp;不过需要注意的是，平均法并不是只能针对回归问题的预测结果进行平均，对于分类问题，有时候我们也可以对分类问题的概率预测结果进行平均，即计算每个样本不同模型的预测概率平均值，然后根据给定阈值计算该样本最终类别，这种方法也被称作概率均值法，例如在如下预测概率的假设情况下，概率均值法有如下计算过程：

<center><img src="https://s2.loli.net/2022/05/16/yWYZEbfmRUntxjo.png" alt="image-20220516173712093" style="zoom:33%;" />

这里需要注意，在某些时候，投票法和概率均值法可能得到不同的结果，例如上述样本2，在投票法计算时最终结果为0类，而在概率均值融合时预测结果为1，这其实是由于0类样本概率较高（接近0.5）且1类样本概率趋近于1导致的。

&emsp;&emsp;在跟多时候，概率均值法也被称作软投票法Soft Voting，对应的，基于类别的投票法则被称为硬投票法Hard Voting。而从理论角度出发，软投票的性能理论上限与硬投票类似。

### 2.基础方法实践

&emsp;&emsp;接下来开始尝试借助上述模型进行投票法和平均法的实践。相关融合方法可以手动实现也可以借助sklearn中的评估器来实现，我们先尝试手动实现，然后再介绍调用sklearn评估器进行融合的方法。

#### 2.1 手动实现过程

&emsp;&emsp;由于当前数据集是分类问题，因此可以考虑围绕上述三个模型进行投票法融合，或者概率平均法进行融合。当然如果是回归问题，也可以参照这里的概率平均法进行平均法融合操作。

&emsp;&emsp;首先分别记录三个模型在训练集和测试集上的预测结果与概率预测结果，为了更加清晰的展示相关过程，这里我们先将每个结果单独保存，在大家熟悉了这个过程后，我们可以考虑直接将所有同类结果保存在一个DataFrame中。

In [18]:
# 训练集上的预测结果
train_prediction1 = logistic_search.best_estimator_.predict(X_train_OE)
train_prediction2 = tree_model.predict(X_train_OE)
train_prediction3 = RF_0.predict(X_train_OE)

# 训练集上的预测概率(预测为1的概率)
train_prediction1_proba = logistic_search.best_estimator_.predict_proba(X_train_OE)[:, 1]
train_prediction2_proba = tree_model.predict_proba(X_train_OE)[:, 1]
train_prediction3_proba = RF_0.predict_proba(X_train_OE)[:, 1]

# 测试集上的预测结果
test_prediction1 = logistic_search.best_estimator_.predict(X_test_OE)
test_prediction2 = tree_model.predict(X_test_OE)
test_prediction3 = RF_0.predict(X_test_OE)

# 测试集上的预测概率
test_prediction1_proba = logistic_search.best_estimator_.predict_proba(X_test_OE)[:, 1]
test_prediction2_proba = tree_model.predict_proba(X_test_OE)[:, 1]
test_prediction3_proba = RF_0.predict_proba(X_test_OE)[:, 1]

In [19]:
RF_0.predict_proba(X_train_OE)

array([[0.99327033, 0.00672967],
       [0.48125643, 0.51874357],
       [0.89506715, 0.10493285],
       ...,
       [0.39270528, 0.60729472],
       [0.9843395 , 0.0156605 ],
       [0.99464364, 0.00535636]])

&emsp;&emsp;然后先尝试投票法（硬投票法）进行融合，首先计算训练集上的投票结果，这里我们可以计算累计票数，也可以计算平均票数，例如累计票数的计算与判别结果如下：

In [20]:
# 大于等于两个评估器预测结果为1，则最终投票结果为1
Voting_train_hard = ((train_prediction1 + 
                      train_prediction2 + 
                      train_prediction3) >= 2) * 1

In [21]:
Voting_train_hard

array([0, 1, 0, ..., 1, 0, 0])

In [22]:
Voting_test_hard = ((test_prediction1 + 
                     test_prediction2 + 
                     test_prediction3) >= 2) * 1

In [23]:
Voting_test_hard

array([0, 0, 0, ..., 0, 1, 0])

然后进一步计算准确率：

In [24]:
accuracy_score(Voting_train_hard, y_train), accuracy_score(Voting_test_hard, y_test)

(0.8345323741007195, 0.7910278250993753)

也可以计算平均得票，即用总票数除以分类器个数（也就是3），结果如下：

In [25]:
# 50%及以上的分类器判别为1，则结果为1，反之亦然
Voting_train_hard = (((train_prediction1 + 
                       train_prediction2 + 
                       train_prediction3) / 3) >= 0.5) * 1

Voting_test_hard = (((test_prediction1 + 
                      test_prediction2 + 
                      test_prediction3) / 3) >= 0.5) * 1

accuracy_score(Voting_train_hard, y_train), accuracy_score(Voting_test_hard, y_test)

(0.8345323741007195, 0.7910278250993753)

当然，在当前分类器较为简单的情况下两种结果没有任何区别，不过需要注意的是，在后续加权硬投票的过程中，二者结果会在高精度计算时会有略微差异，这里建议采用后面一种硬投票的计算方式，这也是sklearn中默认的硬投票的计算方法。

&emsp;&emsp;接下来简单对比单模型输出结果，很明显，简单的硬投票法还有一定的效果提升空间。

|Models|train_score|test_score|
|:--:|:--:|:--:|
|logistic_search|0.8123|0.7836|
|tree_model|0.7991|0.7683|
|RF_0|0.8483|0.7955|
|Voting_hard|0.8345|0.7910|

&emsp;&emsp;接下来继续考虑进行概率均值（软投票）方法进行融合：

In [26]:
Voting_train_soft = (((train_prediction1_proba + 
                       train_prediction2_proba + 
                       train_prediction3_proba) / 3) >= 0.5) * 1

In [27]:
Voting_train_soft

array([0, 1, 0, ..., 1, 0, 0])

In [28]:
Voting_test_soft = (((test_prediction1_proba + 
                      test_prediction2_proba + 
                      test_prediction3_proba) / 3) >= 0.5) * 1

In [29]:
Voting_test_soft

array([0, 0, 0, ..., 0, 1, 0])

In [30]:
accuracy_score(Voting_train_soft, y_train), accuracy_score(Voting_test_soft, y_test)

(0.8258235516849678, 0.787052810902896)

能够发现，概率均值的融合结果不如投票法融合的结果：

|Models|train_score|test_score|
|:--:|:--:|:--:|
|logistic_search|0.8123|0.7836|
|tree_model|0.7991|0.7683|
|RF_0|0.8483|0.7955|
|Voting_hard|0.8345|0.7910|
|Voting_soft|0.8258|0.7870

&emsp;&emsp;不难发现，看似更加死板的加权硬投票法却得到了一个更高的评分。这里需要注意的是，尽管从数值层面来看，软投票的数值表现更丰富——都是高精度浮点数运算，并且配如果可以进一步配合阈值移动或者加权融合，则有较大的调优空间。不过呢，硬投票实际上也有自己的优势，其中最大的优势就在于硬投票的过程实际上自带一个将概率转化为投票的过程，而这个过程实际上是非线性的，即概率小于0.5时输出为0、概率大于0.5时输出为1，而这个非线性的过程就极有可能进一步提升最终融合效果。

&emsp;&emsp;在了解了手动实现过程之后，接下来考虑借助sklearn实现上述过程。

#### 2.2 sklearn实现过程

&emsp;&emsp;在sklearn中，我们可以在ensemble模块内找到投票法评估器。并且根据sklearn的一贯设置，会根据分类问题和回归问题的不同设置不同的评估器，当前问题是分类问题，因此我们需要导入VotingClassifier评估器：

In [31]:
from sklearn.ensemble import VotingClassifier

In [32]:
VotingClassifier?

Init signature:
VotingClassifier(
    estimators,
    *,
    voting='hard',
    weights=None,
    n_jobs=None,
    flatten_transform=True,
    verbose=False,
)
Docstring:     
Soft Voting/Majority Rule classifier for unfitted estimators.

Read more in the :ref:`User Guide <voting_classifier>`.

.. versionadded:: 0.17

Parameters
----------
estimators : list of (str, estimator) tuples
    Invoking the ``fit`` method on the ``VotingClassifier`` will fit clones
    of those original estimators that will be stored in the class attribute
    ``self.estimators_``. An estimator can be set to ``'drop'``
    using ``set_params``.

    .. versionchanged:: 0.21
        ``'drop'`` is accepted. Using None was deprecated in 0.22 and
        support was removed in 0.24.

voting : {'hard', 'soft'}, default='hard'
    If 'hard', uses predicted class labels for majority rule voting.
    Else if 'soft', predicts the class label based on the argmax of
    the sums of the predicted probabilities, which is 

评估器核心参数解释如下：

|Name|Description|   
|:--:|:--:| 
|estimators|由（评估器名称，评估器）所组成的列表|
|voting|融合的方式，包括此前介绍的硬投票和软投票两种|
|weights|融合过程中各评估器的权重|
|flatten_transform|在软投票时打印结果方式|

&emsp;&emsp;其中关于如何进行加权投票，稍后我们会详细讨论，这里先快速实践VotingClassifier的使用方法。

&emsp;&emsp;首先将参与融合的几个模型放在一个列表中：

In [33]:
estimators = [('lr', logistic_search.best_estimator_), ('tree', tree_model), ('rf', RF_0)]

这里有个细节需要注意，在使用sklearn的评估器进行投票法融合时，需要将之前的网格搜索评估器转化为模型评估器，否则后续VotingClassifier在执行fit时，会完整执行一遍网格搜索评估器的搜索过程。

> 注意这个自定义评估器、pipline、网格搜索评估器和投票法评估器相互嵌套的过程，以后会经常用到。

&emsp;&emsp;然后实例化投票法评估器，先尝试执行硬投票。和所有其他的评估器一样，VotingClassifier采集数据的投票结果是基于一个fit过程：

In [34]:
VC_hard = VotingClassifier(estimators).fit(X_train_OE, y_train)

然后直接在给定数据集上进行predicate即可输出预测结果：

In [35]:
VC_hard.predict(X_train_OE)

array([0, 1, 0, ..., 1, 0, 0], dtype=int64)

注意，这里的predicate实际上就是每个模型在训练集上训练完成后，对训练数据进行预测然后硬投票之后的结果，该结果和手动计算结果基本一致：

In [36]:
VC_hard.score(X_train_OE, y_train), VC_hard.score(X_test_OE, y_test)

(0.8345323741007195, 0.7910278250993753)

&emsp;&emsp;当然，我们也可以尝试令其执行软投票过程，只需要将voting参数改为soft即可：

In [37]:
VC_soft = VotingClassifier(estimators, voting='soft').fit(X_train_OE, y_train)

In [38]:
VC_soft.predict(X_train_OE)

array([0, 1, 0, ..., 1, 0, 0], dtype=int64)

In [39]:
# 只有在软投票情况下，才能输出概率预测结果
VC_soft.predict_proba(X_train_OE)

array([[0.98627385, 0.01372615],
       [0.38954929, 0.61045071],
       [0.84200955, 0.15799045],
       ...,
       [0.42045843, 0.57954157],
       [0.93886922, 0.06113078],
       [0.98874499, 0.01125501]])

In [40]:
VC_soft.score(X_train_OE, y_train), VC_soft.score(X_test_OE, y_test)

(0.8258235516849678, 0.787052810902896)

最终结果对比如下：

|Models|train_score|test_score|
|:--:|:--:|:--:|
|logistic_search|0.8123|0.7836|
|tree_model|0.7991|0.7683|
|RF_0|0.8483|0.7955|
|Voting_hard|0.8345|0.7910|
|Voting_soft|0.8258|0.7870

&emsp;&emsp;总的来说，当我们发现投票法的融合结果并不理想、想进一步提升融合效果时，一般来说有两种思路：其一是在多样性层面进行更进一步的优化，例如进一步优化模型多样性——更换效果更好同时（相互之间）更加独立的模型进行融合，或者优化样本多样性和特征多样性——不同模型考虑带入不同数据和不同特征进行训练，等等；其二则是在融合方法上进行优化，哪怕是在投票法的范畴，也有一些更加复杂的融合方法，例如几何平均数投票、加权投票、基于优化器的加权投票、两阶段建模等等方法。本节我们重点探讨模型融合方法层面的内容，下一小节再统一讨论如何构建多样性。

## 二、投票法&平均法的改进策略

&emsp;&emsp;接下来进一步讨论投票法&平均法的进阶融合策略。当然，所谓的进阶方法，只是在融合的方法层面更加复杂，但正如开篇所言，对于模型融合来说，没有哪种方法一定有效，并且也不是融合方法越复杂结果就越好。不过在方法学习阶段，我们应该尽可能掌握更多的方法更多的思路，然后在实践阶段尽可能多的尝试更多方法、组合更多方法，以期获得一个更好的结果。

### 1.几何平均数融合方法 

&emsp;&emsp;首先我们可以将平均法求均值的过程，改为求几何平均数的过程。这是一种非常简单改进策略，关于算术平均和几何平均的数学计算过程如下：

$$算术平均数：\bar x = \frac{x_1+x_2+x_3+...+x_n}{n}$$

$$几何平均数：G_n = \sqrt[n]{x_1*x_2*x_3*...*x_n}$$

&emsp;&emsp;需要注意的是，几何平均数也是很多场景下求均值的最佳方法，例如复利下的平均利率、平均发展速度等，因此在很多时候我们也可以尝试通过几何平均求均值。上述三个模型的几何平均的平均概率结果如下：

In [41]:
np.power(train_prediction1_proba * train_prediction2_proba * train_prediction3_proba, 1/3)

array([0.01195754, 0.60273938, 0.15058701, ..., 0.57184631, 0.04580045,
       0.0083983 ])

In [42]:
Voting_train_GN = (np.power(train_prediction1_proba * 
                            train_prediction2_proba * 
                            train_prediction3_proba, 1/3) >= 0.5) * 1

Voting_test_GN = (np.power(test_prediction1_proba * 
                           test_prediction2_proba * 
                           test_prediction3_proba, 1/3) >= 0.5) * 1

In [43]:
accuracy_score(Voting_train_GN, y_train), accuracy_score(Voting_test_GN, y_test)

(0.8237410071942446, 0.7842135150482681)

能够看出，融合效果并没有任何提升，不过这也只是该方法在当前数据集和当前模型上的表现结果，考虑到该方法实现成本较低，因此在很多情况下都可以快速的进行尝试。事实上，该方法也的确是很多竞赛队伍在模型融合阶段第一阶段会尝试的方法。

### 2.排序平均法

&emsp;&emsp;另外还有一种平均法的改进策略——排序平均法。可以说这是一种转为提升ROC-AUC指标量身定制的方法，基本流程如下：

<center><img src="https://s2.loli.net/2022/05/22/vyw453SGoJWF91c.png" alt="image-20220522154322226" style="zoom:33%;" />

### 3.绝对多数票法

&emsp;&emsp;在投票法范畴，除了简单投票外，还有一种更加严格的投票方法——绝对多数票法。不同于投票法的少数服从多数，绝对多数票法要求票数达到某个临界值后才能输出结果，而如果没达到这个临界值，则不进行预测、而是暂时以某个其他的数值对结果进行标记。当然这个临界值肯定是多于半数票的，例如，假设我们要求只有取得3票以上才能输出结果，则投票法中的极简示例将输出如下结果：

<center><img src="https://s2.loli.net/2022/05/20/HOzePD4T8hbSAmK.png" alt="image-20220520154348748" style="zoom:33%;" />

其中只有2号样本输出了0值预测结果，其他样本都暂时标记为-1，即无法判别。

&emsp;&emsp;一般来说，为了方便区分，我们也会将遵循少数服从多数的投票法称为相对多数票法，在3个分类器的情况下，相对多数票其实就等价于临界值为2的绝对多数票法，而如上所示，如果我们提高这个临界值，则会直接导致两个结果：其一是输出的判别结果更加可信；但同时也将输出大量无法判别类别的样本。如何对剩余样本进行进一步的类别预测？一般来说有两种思路，其一是单独带入这些不确定的样本进行模型训练，然后再进行融合；其二则是采用别的融合方法在现有模型基础上进行预测。而无论采用哪种策略，其实本质上都是一种两阶段建模的方法，这也是模型融合中较为进阶的内容。

> 把不太确定类别的样本单独筛选出来再进行建模，其实非常类似于Boosting的思想。

### 4.阈值移动

&emsp;&emsp;如果说有什么方法一定能够在投票&平均法基础上提升融合效果，那么一定阈值移动一定榜上有名。所谓阈值移动，其实就是改变判别样本类别时的阈值。此前无论是平均法还是几何平均法，在最终将平均后的概率转化为类别预测结果时，其实都是以0.5为阈值进行的判别——当平均后的概率大于0.5时判别为1类、小于0.5时判别为0类。但实际上这个阈值是可以改变的，就像此前逻辑回归模型优化时阈值移动一样，很多时候我们甚至可以将其视作一个超参数来进行优化。并且，相比后面要讨论的加权平均，阈值移动的过拟合风险更小，是融合阶段必不可少的尝试方法。

#### 4.1 阈值移动的简单测试

&emsp;&emsp;一般来说，最佳阈值会在0.5附近，因此我们设置一组0.48、0.5和0.52作为阈值来计算训练集和测试集上的融合准确率：

In [44]:
for thr in [0.48, 0.5, 0.52]:

    Voting_train_soft_thr = (((train_prediction1_proba + 
                               train_prediction2_proba + 
                               train_prediction3_proba) / 3) >= thr) * 1
    
    train_acc = accuracy_score(Voting_train_soft_thr, y_train)
    
    Voting_test_soft_thr = (((test_prediction1_proba + 
                              test_prediction2_proba + 
                              test_prediction3_proba) / 3) >= thr) * 1
    
    test_acc = accuracy_score(Voting_test_soft_thr, y_test)
    
    print("threshold %0.2f:" % thr)
    print("train_Accuracy %0.10f" % train_acc)
    print("test_Accuracy %0.10f" % test_acc)

threshold 0.48:
train_Accuracy 0.8263915184
test_Accuracy 0.7904599659
threshold 0.50:
train_Accuracy 0.8258235517
test_Accuracy 0.7870528109
threshold 0.52:
train_Accuracy 0.8226050738
test_Accuracy 0.7813742192


从上面结果我们不难发现，相比0.5，阈值取为0.48时训练集得到的准确率更高，该结果能够说明两点，其一是阈值移动确实能够提升融合效果，其二则是我们发现训练集和测试集表现出了同步变化的趋势，也就说明阈值移动的过拟合风险较少。

> 需要注意的是，尽管目前该问题还不明显，但实际上控制过拟合其实是所有模型融合优化方法最核心的问题。

> 一般来说，最佳阈值都不会低于0.4也不会超过0.6，为何最佳阈值会在0.5附近，稍后在介绍投票法理论依据时一并介绍。

#### 4.2 借助hyperopt搜索最优阈值

- hyperopt优化器使用方法回顾

&emsp;&emsp;当我们发现了阈值移动可以提升融合效果之后，接下来的问题就是，0.48是否就是最佳阈值呢？还是0.475或者0.478效果更好呢？当然这里可以一个个试，但更推荐的做法是将其视作超参数，带入到优化器当中进行搜索。由于课程已经进展到后期，熟练的组合不同工具来进行使用也是算法工程人员必备技能，因此这里我们考虑组合一个优化器来进行阈值搜索。这里的阈值是一个0到1之间的连续变量，因此可以考虑使用随机网格搜索，但考虑到随机网格搜索的准确性，这里更推荐使用hyperopt进行TPE搜索，能同时保证效果和效率。

In [45]:
from hyperopt import hp, fmin, tpe
from numpy.random import RandomState

&emsp;&emsp;hyperopt的使用需要设置超参数空间、创建目标函数以及创建搜索函数，首先是定义超参数空间，根据经验，大多数情况下最佳阈值都在0.4-0.6之间，因此可以据此设置超参数空间，当然这里设置一个0-1的空间也是可以的。

In [46]:
# 定义超参数空间
params_space = {'thr': hp.uniform("thr",0.4,0.6)}

然后是定义目标函数，这里需要注意，由于hyperopt只能搜索最小值，因此如果我们希望准确率越高，在使用hyperopt进行搜索时需要将目标等价转化为搜索负准确率越小：

In [47]:
# 定义目标函数
def hyperopt_objective(params):
    thr = params['thr']
    
    Voting_train_soft_thr = (((train_prediction1_proba + 
                               train_prediction2_proba + 
                               train_prediction3_proba) / 3) >= thr) * 1
    
    train_acc = accuracy_score(Voting_train_soft_thr, y_train)
    
    return -train_acc

最后则是优化函数，同时关联目标函数和搜索空间，并设置搜索过程所需相关参数：

In [48]:
def param_hyperopt(max_evals):
    params_best = fmin(hyperopt_objective,
                       space=params_space,
                       algo=tpe.suggest,
                       max_evals=max_evals, 
                       rstate=np.random.default_rng(17))    
    return params_best

In [49]:
params_best = param_hyperopt(max_evals=3000) 

100%|██████████████████████████████████████████| 3000/3000 [00:15<00:00, 193.85trial/s, best loss: -0.8271488072699735]


In [50]:
params_best['thr']

0.4787137835346912

In [51]:
Voting_test_soft_thr = (((test_prediction1_proba + 
                          test_prediction2_proba + 
                          test_prediction3_proba) / 3) >= params_best['thr']) * 1
    
test_acc = accuracy_score(Voting_test_soft_thr, y_test)

test_acc

0.7904599659284497

&emsp;&emsp;能够发现，最终搜索得到结果是阈值取值为0.4787时最优，此时训练集准确率为0.8271，测试集准确率为0.7904，相比阈值为0.48时训练集上结果略有提升，尽管测试集目前并没有提升，但从流程上来看，借助优化器来进行搜索肯定比手动验算更准确更高效，该方法也是我们必须要掌握的方法。

- 阈值移动效果的上限测试

&emsp;&emsp;接下来我们进行一个略显“犯规”的操作，即带入测试集进行阈值移动搜索。通常来说测试集的数据信息是不允许在模型训练阶段被泄露的，但这里尝试在测试集上进行最佳阈值搜索，实际上是希望“探探底”，看看阈值移动能在测试集上取得的最好成绩是多少，也就是的测试下阈值移动在测试集上的效果上限。

&emsp;&emsp;该操作过程也非常简单，我们只需要把训练集的预测结果改为测试集的预测结果带入进行搜索即可：

In [52]:
# 定义超参数空间
params_space = {'thr': hp.uniform("thr",0.4,0.6)}

# 定义目标函数
def hyperopt_objective_test(params):
    thr = params['thr']
    
    Voting_test_soft_thr = (((test_prediction1_proba + 
                              test_prediction2_proba + 
                              test_prediction3_proba) / 3) >= thr) * 1

    test_acc = accuracy_score(Voting_test_soft_thr, y_test)
    
    return -test_acc

# 定义优化函数
def param_hyperopt(max_evals):
    params_best = fmin(hyperopt_objective_test,
                       space=params_space,
                       algo=tpe.suggest,
                       max_evals=max_evals, 
                       rstate=np.random.default_rng(17))    
    return params_best

In [53]:
params_best = param_hyperopt(max_evals=3000) 

100%|██████████████████████████████████████████| 3000/3000 [00:15<00:00, 197.30trial/s, best loss: -0.7915956842703009]


In [54]:
params_best

{'thr': 0.4714998631942625}

&emsp;&emsp;能够发现，在测试集上搜索的最佳阈值为0.4714，测试集的最佳得分0.7915，这其实就是阈值移动的效果上限了。该搜索结果和在训练集上搜索得到的结果并不一致，而在训练集上无法输出0.4714的原因也很简单，在训练集上阈值为0.4714时准确率不如0.4787：

In [55]:
# 训练集得分
Voting_train_soft_thr = (((train_prediction1_proba + 
                           train_prediction2_proba + 
                           train_prediction3_proba) / 3) >= params_best['thr']) * 1
    
train_acc = accuracy_score(Voting_train_soft_thr, y_train)

train_acc

0.8254449072321091

两次不同的搜索结果如下：

|搜索方法|thr|train_score|test_score|
|:--:|:--:|:--:|:--:|
|带入训练集搜索|0.4787|<font color="brown">**0.8271**</font>|0.7904|
|带入测试集搜索|0.4714|0.8254|<font color="brown">**0.7915**</font>|

&emsp;&emsp;两次搜索存在差异其实并不难理解，毕竟带入训练集进行搜索时是以训练集评分最高为目标输出的结果，而带入测试集进行搜索时是以测试集评分最高位目标输出最终结果。但是，我们其实更希望二者差异尽可能的减少，即能不能找到一个方法，能让我们在带入训练集的搜索结果也同时能在测试集上尽可能逼近0.7915这个上限呢？毕竟真实情况下我们只能带入训练集进行搜索，同时希望这个结果具备泛化能力（在新的数据集上也有很好的表现）。

&emsp;&emsp;其实如果我们将阈值看成是算法的一个超参数，那么上面的问题就等价于是“如何找到一个泛化能力更强的超参数”。那么应该怎么做呢？熟悉网格搜索+CV过程的小伙伴肯定对此并不陌生，是的，要提升超参数的泛化能力，最好的方式就是交叉验证。

> 需要说明的是，相比其他方法，阈值移动在不进行交叉验证的情况下过拟合倾向并不严重（训练集结果和验证集结果差别不算特别大），甚至可以说是模型融合中过拟合风险较低的一种方法。

- 借助交叉验证提升阈值搜索的泛化能力

&emsp;&emsp;有了基本思路之后，接下来就要考虑如何实践了。首先补充介绍sklearn中的使用进行交叉验证评分的实用函数cross_val_score。

In [56]:
from sklearn.model_selection import cross_val_score

In [57]:
cross_val_score?

Signature:
cross_val_score(
    estimator,
    X,
    y=None,
    *,
    groups=None,
    scoring=None,
    cv=None,
    n_jobs=None,
    verbose=0,
    fit_params=None,
    pre_dispatch='2*n_jobs',
    error_score=nan,
)
Docstring:
Evaluate a score by cross-validation.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
estimator : estimator object implementing 'fit'
    The object to use to fit the data.

X : array-like of shape (n_samples, n_features)
    The data to fit. Can be for example a list, or an array.

y : array-like of shape (n_samples,) or (n_samples, n_outputs),             default=None
    The target variable to try to predict in the case of
    supervised learning.

groups : array-like of shape (n_samples,), default=None
    Group labels for the samples used while splitting the dataset into
    train/test set. Only used in conjunction with a "Group" :term:`cv`
    instance (e.g., :class:`GroupKFold`).

scoring : str or callable, default=None

顾名思义，该函数的核心用途是对输入的X、y进行训练集和验证集的划分，然后令输入的评估器在训练集上训练、在验证集上输出验证结果，例如一个简单的5折交叉验证过程如下：

<center><img src="https://s2.loli.net/2022/05/18/1UAlpgwt2mSbTQ5.png" alt="image-20220518205529331" style="zoom:33%;" />

此外，cross_val_score的关键参数解释如下：

|Name|Description|   
|:--:|:--:| 
|estimator|进行交叉验证的评估器|
|X|交叉验证训练数据特征|
|y|交叉验证训练数据标签|
|scoring|模型评估指标|
|cv|交叉验证折数|
|fit_params|传入评估器的参数|

&emsp;&emsp;例如，我们可以借助cross_val_score函数，非常便捷的输出某个评估器五折交叉验证的结果：

In [58]:
tree = DecisionTreeClassifier()
cross_val_score(tree, X_train_OE, y_train)

array([0.75307474, 0.73226112, 0.72443182, 0.73579545, 0.73674242])

In [59]:
# 输出验证集的平均得分
cross_val_score(tree, X_train_OE, y_train).mean()

0.7360812476706518

In [60]:
# 验证集得分和测试集非常接近
tree.fit(X_train_OE, y_train)
accuracy_score(tree.predict(X_test_OE), y_test)

0.7211811470755253

这里有个细节需要注意，tree只进行了实例化，并没有fit，但最终还是输出了验证集上的准确率结果，说明在该模型在cross_val_score内部完成了训练，只不过这个过程没有显式的fit。

&emsp;&emsp;同时我们知道，VotingClassifier其实本质上也是个评估器，也是可以带入到cross_val_score当中进行计算的，因此一个完整的算法流程浮出水面：我们考虑将VotingClassifier封装到cross_val_score内，并输出交叉验证的平均结果，同时以该结果作为目标函数，进行TPE阈值搜索，此时就相当于手动创建了一个基于验证集的平均得分来进行超参数搜索的过程。这里唯一需要注意的是，VotingClassifier唯一的问题在于无法在predict方法里面调整阈值，我们需要手动封装一个能够实现阈值调整的VotingClassifier_threshold评估器。该评估器的封装过程和阈值移动逻辑回归评估器封装过程非常类似，只需要给出关键参数接口、同时在定义predict时设置根据阈值参数输出结果即可：

In [61]:
class VotingClassifier_threshold(BaseEstimator, ClassifierMixin, TransformerMixin):
    
    def __init__(self, estimators, voting='hard', weights=None, thr=0.5):
        self.estimators = estimators
        self.voting = voting
        self.weights = weights
        self.thr = thr
        
    def fit(self, X, y):
        VC = VotingClassifier(estimators = self.estimators, 
                              voting = self.voting, 
                              weights = self.weights)
        
        VC.fit(X, y)
        self.clf = VC
        
        return self
        
    def predict_proba(self, X):
        if self.voting == 'soft':
            res_proba = self.clf.predict_proba(X)
        else:
            res_proba = None
        return res_proba
    
    def predict(self, X):
        if self.voting == 'soft':
            res = (self.clf.predict_proba(X)[:, 1] >= self.thr) * 1
        else:
            res = self.clf.predict(X)
        return res
    
    def score(self, X, y):
        acc = accuracy_score(self.predict(X), y)
        return acc

> 同样，该评估器也需要被写入telcoFunc.py文件中。

简单测试VotingClassifier_threshold的实践效果：

In [62]:
VC_soft_thr = VotingClassifier_threshold(estimators, voting='soft', thr=0.4787).fit(X_train_OE, y_train)
VC_soft_thr.score(X_train_OE, y_train), VC_soft_thr.score(X_test_OE, y_test)

(0.8271488072699735, 0.7904599659284497)

然后尝试将VC_soft_thr作为estimator传入cross_val_score，计算验证集上的得分：

In [63]:
# 每个验证集上的得分
cross_val_score(VC_soft_thr, X_train_OE, y_train, cv=5)

array([0.82213813, 0.79659413, 0.80681818, 0.81155303, 0.80492424])

In [64]:
# 验证集上的平均得分
cross_val_score(VC_soft_thr, X_train_OE, y_train, cv=5).mean()

0.8084055431323642

能够发现，相比训练集上的得分，验证集上的平均得分和测试集得分更为接近。这也更加坚定了我们的判断：以验证集的平均得分作为目标函数进行搜索能获得一个泛化能力更强的结果。

&emsp;&emsp;在一切准备妥当之后，接下来考虑以验证集的平均得分作为目标函数进行TPE搜索，基本过程如下：

In [65]:
# 定义超参数空间
params_space = {'thr': hp.uniform("thr", 0.4, 0.6)}

# 定义目标函数
def hyperopt_objective_val(params):
    thr = params['thr']
    
    # 创建带阈值的平均法评估器
    VC_soft_thr = VotingClassifier_threshold(estimators, voting='soft', thr=thr)

    # 输出验证集上的平均得分
    val_score = cross_val_score(VC_soft_thr, 
                                X_train_OE, 
                                y_train, 
                                scoring='accuracy', 
                                n_jobs=15,
                                cv=3).mean()
    
    return -val_score

# 定义优化函数
def param_hyperopt_val(max_evals):
    params_best = fmin(hyperopt_objective_val,
                       space=params_space,
                       algo=tpe.suggest,
                       max_queue_len=5,
                       max_evals=max_evals, 
                       rstate=np.random.default_rng(17))    
    return params_best

这里有两点需要注意，其一是可以在cross_val_score中设置一个n_jobs以提高每次验证集划分、训练、评分的速度，其二则是关于交叉验证折数划分的问题，这里建议划分三折，主要原因也是考虑到当前训练数据的数据总量，划分折数过多容易导致验证集数量太少，从而导致验证集评分方差过大，结果可信度降低。

In [66]:
params_best_val = param_hyperopt_val(max_evals=3000) 

100%|███████████████████████████████████████████| 3000/3000 [11:08<00:00,  4.49trial/s, best loss: -0.8085965059453134]


In [67]:
params_best_val

{'thr': 0.46991159868237964}

能够发现，搜索出来的阈值结果会更加接近测试集上搜索出来的阈值最优解0.4714，接下来查看在该阈值情况下训练集和测试集的得分：

In [68]:
VC_soft_thr = VotingClassifier_threshold(estimators, 
                                         voting='soft', 
                                         thr=params_best_val['thr']).fit(X_train_OE, y_train)

In [69]:
VC_soft_thr.score(X_train_OE, y_train), VC_soft_thr.score(X_test_OE, y_test)

(0.8258235516849678, 0.7910278250993753)

能够发现，加入交叉验证之后，训练集和测试集的结果一致性得到了显著加强，相比只带入训练集而不进行交叉验证的搜索过程，本轮搜索得到了一个更好的结果，阈值的泛化能力进一步增强，在训练集上训练的结果到达了0.7910，能够看出交叉验证对于搜索结果泛化能力的提升还是较为明显的。虽然本次实验没有达到测试集效果上限，但这其实是也是一般情况，在很多时候，这个上限只能逼近但无法达到，很多时候交叉验证的引入能显著提升搜索结果的泛化能力，但训练数据和测试数据之间总是会存在无法捕捉且不一致的规律，我们要做的（能做的），就是取不断的提高泛化能力、取逼近这个上限。

|搜索方法|thr|train_score|test_score|
|:--:|:--:|:--:|:--:|
|带入训练集搜索|0.4787|<font color="brown">**0.8271**</font>|0.7904|
|带入测试集搜索|0.4714|0.8254|<font color="brown">**0.7915**</font>|
|训练集+CV搜索|0.4699|0.8258|<font color="brown">**0.7910**</font>|

> 需要注意的是，带入交叉验证与提高模型泛化能力实际上也是一个概率问题，只能说大概率下引入交叉验证后能提升泛化能力，但并非100%能提高泛化能力，这和数据、模型都有很大的关系。

> 另外，这里我们说在测试集上的搜索结果是阈值调整的效果上限，实际上这个上限只是实践层面的上限，并不是理论层面的上限，其实投票法&平均法（包括阈值移动）的理论效果上限是非常高的。因此，这个实践层面的上限，我们也可以称其为软上限，而理论层面的上限，可以称其为硬上限。

- 硬投票的阈值移动

&emsp;&emsp;关于硬投票能否进行阈值移动，当然也是可以的，不过对于一个二分类问题，在三个模型参与融合的情况下，每个样本的的票只可能在0-3之间，因此阈值调整空间有限，所需要